In [31]:
import os
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
from tqdm.notebook import tqdm
import pandas as pd

In [32]:
path = './reports'

pdf_filepaths = []
for root, directories, files in os.walk(path, topdown=False):
	for name in files:
		if name[-4:] == '.pdf':
			pdf_filepaths.append(os.path.join(root, name))

reports = pd.DataFrame(pdf_filepaths, columns = ['filepath'])
reports.head()

,filepath
0,./reports/entire_merck_crr19.pdf


# OCR

In [33]:
# https://towardsdatascience.com/extracting-text-from-scanned-pdf-using-pytesseract-open-cv-cd670ee38052
# https://pdf2image.readthedocs.io/en/latest/reference.html
# this is also getting the page number because of performance reasons
def save_as_image(filepath):
    pages = convert_from_path(filepath)
    for p in range(len(pages)):
        path = filepath[:-4] + '_' + str(p) + '.jpg'
        # only save if file does not exist
        if (os.path.isfile(path) == False):
            pages[p].save(path, 'JPEG')
    return len(pages)

if (True):
    reports['number_of_pages'] = reports['filepath'].apply(lambda fp: save_as_image(fp))

In [34]:
from multiprocessing import cpu_count, Pool
from pytesseract import image_to_string
import numpy as np

def text_from_ocr(df) -> pd.DataFrame:
    rows = len(df)
    #for index, row in df.iterrows():
    for index, row in tqdm(df.iterrows()):
        print(str(index)+'/'+str(rows) + ' Pdf')
        text = ''
        pages = row['number_of_pages']
        for page in range(pages):
            print(str(page)+'/'+ str(pages) + 'Pages')
            text += image_to_string(row['filepath'][:-4] + '_' + str(page) + '.jpg')
        df.loc[index, 'text'] = text
        print(index)
    return df

# https://towardsdatascience.com/make-your-own-super-pandas-using-multiproc-1c04f41944a1#6028
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

#df_unique = pd.read_csv('captum.csv')
reports = parallelize_dataframe(reports, text_from_ocr, len(os.sched_getaffinity(0)))
reports

ltiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "<ipython-input-34-3e84c606527a>", line 13, in text_from_ocr
    text += image_to_string(row['filepath'][:-4] + '_' + str(page) + '.jpg')
  File "/home/fiete/Documents/studium/fuehrung_und_nachhaltigkeit/nlp_env/lib/python3.8/site-packages/pytesseract/pytesseract.py", line 409, in image_to_string
    return {
  File "/home/fiete/Documents/studium/fuehrung_und_nachhaltigkeit/nlp_env/lib/python3.8/site-packages/pytesseract/pytesseract.py", line 412, in <lambda>
    Output.STRING: lambda: run_and_get_output(*args),
  File "/home/fiete/Documents/studium/fuehrung_und_nachhaltigkeit/nlp_env/lib/python3.8/site-packages/pytesseract/pytesseract.py", line 287, in run_and_get_output
    run_tesseract(**kwargs)
  File "/home/fiete/Documents/studium/fuehrung_und_nachhaltigkeit/nlp_env/l

KeyboardInterrupt: 

In [25]:
reports.to_csv('reports.csv')